In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(OPENAI_API_KEY[:2])
print(UPSTAGE_API_KEY[30:])
print(TAVILY_API_KEY[:2])

sk
YN
tv


In [2]:
import operator
import re
from typing import TypedDict, Annotated, List

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.documents import Document
from langchain_community.embeddings import FakeEmbeddings
from langchain_community.vectorstores import Chroma

# (LangGraph 설치 필요)
# from langgraph.graph import StateGraph, START
# from langgraph.graph.message import MessageState
# from langgraph.checkpoint.memory import MemoryCheckpoint

In [3]:
# 더미 카페 메뉴 데이터
CAFE_MENU_DATA = [
    ("아메리카노", "신선한 원두로 내린 기본 커피. 깔끔하고 진한 맛. ₩4,500.", {"type": "커피", "is_recommend": False}),
    ("바닐라 라떼", "달콤한 바닐라 시럽이 들어간 부드러운 라떼. 인기 메뉴. ₩5,500.", {"type": "라떼", "is_recommend": True}),
    ("콜드 브루", "찬물로 오랜 시간 추출한 커피. 부드럽고 산미가 특징. ₩5,000.", {"type": "커피", "is_recommend": True}),
    ("딸기 스무디", "신선한 딸기를 갈아 만든 시원한 음료. ₩6,000.", {"type": "스무디", "is_recommend": False}),
    ("초코 케이크", "진한 초콜릿 맛의 케이크. 디저트 추천 메뉴. ₩7,500.", {"type": "디저트", "is_recommend": True}),
]

# Vector DB 생성 (FakeEmbeddings 사용)
docs = [
    Document(page_content=desc, metadata={"menu_name": name, **meta})
    for name, desc, meta in CAFE_MENU_DATA
]

# 실제 구현 시 OpenAI, HuggingFace 등의 임베딩 모델 사용
vectorstore = Chroma.from_documents(
    docs, 
    FakeEmbeddings(size=1024), 
    collection_name="cafe_menu_db"
)

In [4]:
class AgentState(TypedDict):
    """LangGraph의 상태 정의"""
    # MessageState의 핵심: 대화 이력을 자동으로 관리
    messages: Annotated[List[BaseMessage], operator.add]
    # 추가 필드
    query_type: str # 'menu_query', 'price_query', 'recommendation'

In [5]:
def classify_query(state: AgentState) -> AgentState:
    """사용자 문의를 키워드 기반으로 분류합니다."""
    last_message = state["messages"][-1].content.lower()
    
    query_type = "menu_query" # 기본값

    # 1. 가격 문의
    if any(keyword in last_message for keyword in ["가격", "얼마", "비용"]):
        query_type = "price_query"
    
    # 2. 추천 요청
    elif any(keyword in last_message for keyword in ["추천", "뭐가 좋아", "인기 있는"]):
        query_type = "recommendation"
        
    # 3. 메뉴 문의 (나머지)
    elif any(keyword in last_message for keyword in ["메뉴", "있어", "커피", "라떼", "스무디", "케이크"]):
        query_type = "menu_query"
        
    # 4. 기타/인사 (처음에는 단순 메뉴 문의로 처리)
    else:
        query_type = "menu_query"
        
    print(f"DEBUG: 문의 유형: {query_type}")
    return {"query_type": query_type}

In [6]:
def extract_menu_info(doc: Document) -> dict:
    """Vector DB 문서에서 구조화된 메뉴 정보 추출"""
    content = doc.page_content
    menu_name = doc.metadata.get('menu_name', 'Unknown')
    
    # 정규표현식으로 가격, 설명 등 추출
    price_match = re.search(r'₩([\d,]+)', content)
    # page_content의 처음 부분만 설명으로 가정
    description_match = re.search(r'설명:\s*(.+?)(?:\n|$)', content, re.DOTALL) 
    
    # 실제 데이터에서는 '설명: '을 제거하고 내용만 추출하도록 수정
    description = content.replace(f"{menu_name}: ", "").strip()
    
    return {
        "name": menu_name,
        "price": price_match.group(0) if price_match else "가격 정보 없음",
        "description": description
    }

In [7]:
def generate_menu_response(state: AgentState) -> AgentState:
    """특정 메뉴 문의에 대한 응답을 생성합니다."""
    user_message = state["messages"][-1].content
    
    # 전략: 사용자 메시지를 직접 검색어로 활용
    docs = vectorstore.similarity_search(user_message, k=2)

    if not docs:
        response = "죄송합니다. 요청하신 메뉴에 대한 정보를 찾을 수 없습니다. 다른 메뉴를 문의해 주시겠어요?"
    else:
        info = extract_menu_info(docs[0])
        response = f"네, {info['name']}에 대해 문의하셨군요! {info['name']}은 {info['description']}의 특징을 가지며, 가격은 {info['price']}입니다."
        
    return {"messages": [AIMessage(content=response)]}


def generate_price_response(state: AgentState) -> AgentState:
    """가격 문의에 대한 응답을 생성합니다."""
    user_message = state["messages"][-1].content
    
    # 전략 1: 사용자 메시지에서 메뉴명 추출 시도 (간단 구현 생략)
    # 전략 2: 사용자 메시지를 검색어로 사용
    docs = vectorstore.similarity_search(user_message, k=3)

    response_parts = []
    
    if not docs:
        response = "죄송합니다. 가격을 문의하신 메뉴 정보를 찾을 수 없습니다. 메뉴 이름을 알려주시겠어요?"
    else:
        # 검색된 상위 3개 메뉴의 가격 정보 제공
        response_parts.append("문의하신 내용과 관련된 메뉴의 가격 정보입니다:")
        for doc in docs:
            info = extract_menu_info(doc)
            response_parts.append(f"- **{info['name']}**: {info['price']}")
            
        response = "\n".join(response_parts)
        
    return {"messages": [AIMessage(content=response)]}


def generate_recommendation_response(state: AgentState) -> AgentState:
    """메뉴 추천 요청에 대한 응답을 생성합니다."""
    user_message = state["messages"][-1].content
    
    # 전략: 사용자 메시지 + 인기 메뉴 키워드로 검색 후, 추천 메타데이터 활용
    docs = vectorstore.similarity_search(f"{user_message} 인기 메뉴 추천", k=5)
    
    # 추천 플래그가 True인 메뉴만 필터링
    recommendations = [doc for doc in docs if doc.metadata.get('is_recommend', False)]
    
    if not recommendations:
        # Fallback: 모든 메뉴 중 인기 메뉴를 검색 (예: 바닐라 라떼, 콜드 브루)
        recommendations = [
            doc for doc in vectorstore.similarity_search("인기 메뉴", k=5) 
            if doc.metadata.get('is_recommend', False)
        ]

    if not recommendations:
        response = "현재 추천 드릴 만한 인기 메뉴 정보가 없습니다. 기본적인 아메리카노는 어떠세요?"
    else:
        # 상위 2개 추천 메뉴 정보 제공
        top_rec = [extract_menu_info(doc) for doc in recommendations[:2]]
        
        response = "오늘의 특별 추천 메뉴를 알려드릴게요! ✨\n"
        for info in top_rec:
            response += f"- **{info['name']}**: {info['description']} 가격은 {info['price']}입니다.\n"
        response += "이 중에서 선택해 보시겠어요?"
        
    return {"messages": [AIMessage(content=response)]}

In [8]:
def route_query(state: AgentState) -> str:
    """문의 유형에 따라 다음 노드를 결정합니다."""
    return state["query_type"]

In [9]:
from langgraph.graph import StateGraph, END

# 그래프 정의
graph_builder = StateGraph(AgentState)

# 1. 노드 추가
graph_builder.add_node("classify", classify_query)
graph_builder.add_node("menu_res", generate_menu_response)
graph_builder.add_node("price_res", generate_price_response)
graph_builder.add_node("recommend_res", generate_recommendation_response)

# 2. 시작 지점
graph_builder.set_entry_point("classify")

# 3. 조건부 엣지 (라우팅)
graph_builder.add_conditional_edges(
    "classify",  # 소스 노드
    route_query, # 분기 함수
    {            # 분기 결과와 다음 노드 매핑
        "menu_query": "menu_res",
        "price_query": "price_res",
        "recommendation": "recommend_res",
    }
)

# 4. 종료 엣지
graph_builder.add_edge("menu_res", END)
graph_builder.add_edge("price_res", END)
graph_builder.add_edge("recommend_res", END)

# 그래프 컴파일
app = graph_builder.compile()

print("✅ LangGraph 메뉴 추천 시스템이 준비되었습니다.")

✅ LangGraph 메뉴 추천 시스템이 준비되었습니다.


In [10]:
# 사용자 입력 시뮬레이션
def get_ai_response(user_input: str, app_instance):
    """LangGraph를 실행하고 응답을 반환합니다."""
    # 상태 초기화: HumanMessage를 추가하여 시작
    initial_state = AgentState(messages=[HumanMessage(content=user_input)], query_type="")
    
    # LangGraph 실행 (checkpointer를 사용하면 대화 이력 자동 유지 가능)
    final_state = app_instance.invoke(initial_state)
    
    # AI의 마지막 응답 메시지 반환
    return final_state["messages"][-1].content


# ----------------------------------------------------
# 테스트 1: 메뉴 문의 (menu_query)
# ----------------------------------------------------
print("\n--- [TEST 1: 메뉴 문의] ---")
user_input_1 = "콜드 브루가 뭐야?"
response_1 = get_ai_response(user_input_1, app)
print(f"사용자: {user_input_1}")
print(f"AI: {response_1}")
# DEBUG: 문의 유형: menu_query
# AI: 네, 콜드 브루에 대해 문의하셨군요! 찬물로 오랜 시간 추출한 커피. 부드럽고 산미가 특징.의 특징을 가지며, 가격은 ₩5,000입니다.


# ----------------------------------------------------
# 테스트 2: 가격 문의 (price_query)
# ----------------------------------------------------
print("\n--- [TEST 2: 가격 문의] ---")
user_input_2 = "라떼랑 케이크 가격이 얼마야?"
response_2 = get_ai_response(user_input_2, app)
print(f"사용자: {user_input_2}")
print(f"AI: {response_2}")
# DEBUG: 문의 유형: price_query
# AI: 문의하신 내용과 관련된 메뉴의 가격 정보입니다:
# - **초코 케이크**: ₩7,500
# - **바닐라 라떼**: ₩5,500


# ----------------------------------------------------
# 테스트 3: 추천 요청 (recommendation)
# ----------------------------------------------------
print("\n--- [TEST 3: 추천 요청] ---")
user_input_3 = "오늘의 인기 메뉴 추천 좀 해줘"
response_3 = get_ai_response(user_input_3, app)
print(f"사용자: {user_input_3}")
print(f"AI: {response_3}")
# DEBUG: 문의 유형: recommendation
# AI: 오늘의 특별 추천 메뉴를 알려드릴게요! ✨
# - **바닐라 라떼**: 달콤한 바닐라 시럽이 들어간 부드러운 라떼. 인기 메뉴. 가격은 ₩5,500입니다.
# - **콜드 브루**: 찬물로 오랜 시간 추출한 커피. 부드럽고 산미가 특징. 가격은 ₩5,000입니다.
# 이 중에서 선택해 보시겠어요?


--- [TEST 1: 메뉴 문의] ---
DEBUG: 문의 유형: menu_query
사용자: 콜드 브루가 뭐야?
AI: 네, 콜드 브루에 대해 문의하셨군요! 콜드 브루은 찬물로 오랜 시간 추출한 커피. 부드럽고 산미가 특징. ₩5,000.의 특징을 가지며, 가격은 ₩5,000입니다.

--- [TEST 2: 가격 문의] ---
DEBUG: 문의 유형: price_query
사용자: 라떼랑 케이크 가격이 얼마야?
AI: 문의하신 내용과 관련된 메뉴의 가격 정보입니다:
- **콜드 브루**: ₩5,000
- **초코 케이크**: ₩7,500
- **바닐라 라떼**: ₩5,500

--- [TEST 3: 추천 요청] ---
DEBUG: 문의 유형: recommendation
사용자: 오늘의 인기 메뉴 추천 좀 해줘
AI: 오늘의 특별 추천 메뉴를 알려드릴게요! ✨
- **초코 케이크**: 진한 초콜릿 맛의 케이크. 디저트 추천 메뉴. ₩7,500. 가격은 ₩7,500입니다.
- **콜드 브루**: 찬물로 오랜 시간 추출한 커피. 부드럽고 산미가 특징. ₩5,000. 가격은 ₩5,000입니다.
이 중에서 선택해 보시겠어요?
